In [14]:
import os, json
import numpy as np
import tensorflow as tf
import keras_tuner

In [15]:
print("> Uploading test_dataset")
uploaded_test_dataset = tf.data.Dataset.load("/home/bs_fgalavotti/28_01_2025/package/HalfDataset/test_dataset")

print(f'--- BENCHMARK MODEL ---')

# Load the model
model = tf.keras.models.load_model(f'/home/bs_fgalavotti/28_01_2025/package/L2Half/best_models/marcello.keras')

model.summary()

# Accumulators for errors
abs_errors = []
squared_errors = []

# Iterate over the dataset
for batch in uploaded_test_dataset:
    X_test, y_test = batch  # Unpack features and labels
    y_pred = model.predict(X_test, verbose=0)  # Get predictions

    # Calculate errors for the batch
    batch_abs_errors = np.abs(y_pred - y_test)
    batch_squared_errors = (y_pred - y_test) ** 2

    # Append to accumulators
    abs_errors.extend(batch_abs_errors)
    squared_errors.extend(batch_squared_errors)

# Convert lists to NumPy arrays
abs_errors = np.array(abs_errors)
squared_errors = np.array(squared_errors)

# Compute metrics
mae = np.mean(abs_errors)
std_mae = np.std(abs_errors)
mse = np.mean(squared_errors)
std_mse = np.std(squared_errors)

print(f"MAE: {mae:.4f}, Std on MAE: {std_mae:.4f}")
print(f"MSE: {mse:.4f}, Std on MSE: {std_mse:.4f}")

for i in range(10):

    print(f'--- MODEL_{i} ---')

    # Load the model
    model = tf.keras.models.load_model(f'/home/bs_fgalavotti/28_01_2025/package/L2Half/best_models/model_{i}.keras')

    model.summary()

    # Accumulators for errors
    abs_errors = []
    squared_errors = []

    # Iterate over the dataset
    for batch in uploaded_test_dataset:
        X_test, y_test = batch  # Unpack features and labels
        y_pred = model.predict(X_test, verbose=0)  # Get predictions

        # Calculate errors for the batch
        batch_abs_errors = np.abs(y_pred - y_test)
        batch_squared_errors = (y_pred - y_test) ** 2

        # Append to accumulators
        abs_errors.extend(batch_abs_errors)
        squared_errors.extend(batch_squared_errors)

    # Convert lists to NumPy arrays
    abs_errors = np.array(abs_errors)
    squared_errors = np.array(squared_errors)

    # Compute metrics
    mae = np.mean(abs_errors)
    std_mae = np.std(abs_errors)
    mse = np.mean(squared_errors)
    std_mse = np.std(squared_errors)

    print(f"MAE: {mae:.4f}, Std on MAE: {std_mae:.4f}")
    print(f"MSE: {mse:.4f}, Std on MSE: {std_mse:.4f}")

> Uploading test_dataset
--- BENCHMARK MODEL ---
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 50, 16)            384       
                                                                 
 batch_normalization_4 (Bat  (None, 50, 16)            64        
 chNormalization)                                                
                                                                 
 re_lu_4 (ReLU)              (None, 50, 16)            0         
                                                                 
 conv1d_3 (Conv1D)           (None, 50, 16)            768       
                                                                 
 batch_normalization_5 (Bat  (None, 50, 16)            64        
 chNormalization)                                                
                                                                 
 re_lu_

In [16]:
# Number of channels in your y_test (assuming 20 channels)
num_channels = 10

print(f'--- BENCHMARK MODEL ---')

# Import the model
model = tf.keras.models.load_model(f'/home/bs_fgalavotti/28_01_2025/package/L2Half/best_models/marcello.keras')

# Initialize accumulators for errors, one for each channel
abs_errors = np.zeros((num_channels,))  # Stores absolute errors for each channel
squared_errors = np.zeros((num_channels,))  # Stores squared errors for each channel
abs_errors_squared = np.zeros((num_channels,))  # Stores squared absolute errors for std calculation
squared_errors_squared = np.zeros((num_channels,))  # Stores squared squared errors for std calculation

# Accumulators for the count of samples
sample_count = 0

# Iterate over the dataset
for batch in uploaded_test_dataset:
    X_test, y_test = batch  # Unpack features and labels
    y_pred = model.predict(X_test, verbose=0)  # Get predictions

    # For each channel, calculate errors
    for i in range(num_channels):
        channel_abs_errors = np.abs(y_pred[:, i] - y_test[:, i])
        channel_squared_errors = (y_pred[:, i] - y_test[:, i]) ** 2

        # Accumulate the errors for each channel
        abs_errors[i] += np.sum(channel_abs_errors)
        squared_errors[i] += np.sum(channel_squared_errors)

        # Accumulate for standard deviation calculation
        abs_errors_squared[i] += np.sum(channel_abs_errors ** 2)
        squared_errors_squared[i] += np.sum(channel_squared_errors ** 2)
    
    sample_count += X_test.shape[0]  # Count number of samples in the batch

# Normalize by the number of samples to get the mean errors
mae = abs_errors / sample_count
mse = squared_errors / sample_count

# Compute standard deviations for each channel
std_mae = np.sqrt((abs_errors_squared / sample_count) - mae ** 2)
std_mse = np.sqrt((squared_errors_squared / sample_count) - mse ** 2)

# Print results for each channel
for i in range(num_channels):
    print(f"Channel {i}:")
    print(f"  MAE: {mae[i]:.4f}, Std on MAE: {std_mae[i]:.4f}")
    print(f"  MSE: {mse[i]:.4f}, Std on MSE: {std_mse[i]:.4f}")

for j in range(10):
    print(f'--- MODEL_{j} ---')

    # Import the model
    model = tf.keras.models.load_model(f'/home/bs_fgalavotti/28_01_2025/package/L2Half/best_models/model_{j}.keras')
    
    # Initialize accumulators for errors, one for each channel
    abs_errors = np.zeros((num_channels,))  # Stores absolute errors for each channel
    squared_errors = np.zeros((num_channels,))  # Stores squared errors for each channel
    abs_errors_squared = np.zeros((num_channels,))  # Stores squared absolute errors for std calculation
    squared_errors_squared = np.zeros((num_channels,))  # Stores squared squared errors for std calculation

    # Accumulators for the count of samples
    sample_count = 0

    # Iterate over the dataset
    for batch in uploaded_test_dataset:
        X_test, y_test = batch  # Unpack features and labels
        y_pred = model.predict(X_test, verbose=0)  # Get predictions

        # For each channel, calculate errors
        for i in range(num_channels):
            channel_abs_errors = np.abs(y_pred[:, i] - y_test[:, i])
            channel_squared_errors = (y_pred[:, i] - y_test[:, i]) ** 2

            # Accumulate the errors for each channel
            abs_errors[i] += np.sum(channel_abs_errors)
            squared_errors[i] += np.sum(channel_squared_errors)

            # Accumulate for standard deviation calculation
            abs_errors_squared[i] += np.sum(channel_abs_errors ** 2)
            squared_errors_squared[i] += np.sum(channel_squared_errors ** 2)
        
        sample_count += X_test.shape[0]  # Count number of samples in the batch

    # Normalize by the number of samples to get the mean errors
    mae = abs_errors / sample_count
    mse = squared_errors / sample_count

    # Compute standard deviations for each channel
    std_mae = np.sqrt((abs_errors_squared / sample_count) - mae ** 2)
    std_mse = np.sqrt((squared_errors_squared / sample_count) - mse ** 2)

    # Print results for each channel
    for i in range(num_channels):
        print(f"Channel {i}:")
        print(f"  MAE: {mae[i]:.4f}, Std on MAE: {std_mae[i]:.4f}")
        print(f"  MSE: {mse[i]:.4f}, Std on MSE: {std_mse[i]:.4f}")

--- BENCHMARK MODEL ---


Channel 0:
  MAE: 0.4075, Std on MAE: 0.2133
  MSE: 0.2115, Std on MSE: 0.1912
Channel 1:
  MAE: 0.0706, Std on MAE: 0.0640
  MSE: 0.0091, Std on MSE: 0.0150
Channel 2:
  MAE: 0.4244, Std on MAE: 0.2123
  MSE: 0.2252, Std on MSE: 0.1841
Channel 3:
  MAE: 0.0328, Std on MAE: 0.0358
  MSE: 0.0024, Std on MSE: 0.0052
Channel 4:
  MAE: 0.4856, Std on MAE: 0.2111
  MSE: 0.2804, Std on MSE: 0.1926
Channel 5:
  MAE: 0.0273, Std on MAE: 0.0336
  MSE: 0.0019, Std on MSE: 0.0045
Channel 6:
  MAE: 0.5397, Std on MAE: 0.2819
  MSE: 0.3708, Std on MSE: 0.3056
Channel 7:
  MAE: 0.0728, Std on MAE: 0.0680
  MSE: 0.0099, Std on MSE: 0.0203
Channel 8:
  MAE: 0.2436, Std on MAE: 0.1128
  MSE: 0.0721, Std on MSE: 0.0547
Channel 9:
  MAE: 0.2171, Std on MAE: 0.1052
  MSE: 0.0582, Std on MSE: 0.0491
--- MODEL_0 ---
Channel 0:
  MAE: 0.4087, Std on MAE: 0.2098
  MSE: 0.2110, Std on MSE: 0.1871
Channel 1:
  MAE: 0.0734, Std on MAE: 0.0633
  MSE: 0.0094, Std on MSE: 0.0149
Channel 2:
  MAE: 0.4237, Std on MAE